In [14]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit

In [268]:
import warnings
warnings.filterwarnings('ignore')

In [123]:
C1_c = "#649B1C"
C2_c = "#00B593"
C3_c = "#4E84D4"

In [11]:
def f_logistic(x, L ,x0, k):
    y = L-L / (1 + np.exp(-k*(x-x0)))
    return (y)

In [45]:
def f_gompertz(x, L ,x0, k):
    y = L-L*np.exp(np.exp(-k*(x-x0)))
    return (y)

In [299]:
final_energy = pd.read_csv("../committed_facilities/AR6_final_energy_R10.csv")
final_energy["TWh/yr"] = final_energy["EJ/yr"]*277.77
final_energy

,Model,Scenario,Category,Region,Year,EJ/yr,TWh/yr
0,MESSAGEix-GLOBIOM_1.1,EN_NPi2020_600_DR2p,C1,R10REST_ASIA,2044,13.240762,3677.886579
1,REMIND-MAgPIE 2.1-4.2,EN_NPi2020_300f,C1,R10REST_ASIA,2024,7.582120,2106.085472
2,REMIND-MAgPIE 2.1-4.2,EN_NPi2020_300f,C1,R10ROWO,2024,4.881580,1355.956477
3,REMIND-MAgPIE 2.1-4.2,EN_NPi2020_400,C1,R10AFRICA,2024,2.249200,624.760284
4,REMIND-MAgPIE 2.1-4.2,EN_NPi2020_400,C1,R10CHINA+,2024,24.690240,6858.207965
...,...,...,...,...,...,...,...
386814,REMIND 2.1,R2p1_SSP2-PkBudg1100,C3,R10ROWO,2034,6.599640,1833.182003
386815,REMIND 2.1,R2p1_SSP2-PkBudg1100,C3,R10REST_ASIA,2034,12.872180,3575.505439
386816,REMIND 2.1,R2p1_SSP2-PkBudg1100,C3,R10REF_ECON,2034,5.335860,1482.141832
386817,REMIND 2.1,R2p1_SSP2-PkBudg1300,C3,R10INDIA+,2034,13.004760,3612.332185


# Coal growth rate 

In [30]:
coal_data = pd.read_csv("../committed_facilities/AR6_Coal_GW_R10_capacity.csv")
coal_data

,Model,Scenario,Category,Region,Year,CumCoal_GW
0,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,R10NORTH_AM,2049,0.016280
1,REMIND-MAgPIE 1.7-3.0,PEP_1p5C_red_eff,C1,R10AFRICA,2024,1.693100
2,REMIND-MAgPIE 1.7-3.0,PEP_1p5C_red_eff,C1,R10CHINA+,2024,489.107340
3,REMIND-MAgPIE 1.7-3.0,PEP_1p5C_red_eff,C1,R10EUROPE,2024,48.303700
4,REMIND-MAgPIE 1.7-3.0,PEP_1p5C_red_eff,C1,R10INDIA+,2024,171.671180
...,...,...,...,...,...,...
366654,REMIND 2.1,TechCost-SSP2-B1100-ccsH,C3,R10REST_ASIA,2034,52.387460
366655,REMIND 2.1,TechCost-SSP2-B1100-ccsH,C3,R10ROWO,2034,0.084520
366656,REMIND 2.1,TechCost-SSP2-B1100-coalH,C3,R10AFRICA,2034,13.106920
366657,REMIND 2.1,TechCost-SSP2-B1100-bioH,C3,R10REF_ECON,2034,2.171660


In [46]:
coal_data["Region"].unique()

array(['R10NORTH_AM', 'R10AFRICA', 'R10CHINA+', 'R10EUROPE', 'R10INDIA+',
       'R10LATIN_AM', 'R10MIDDLE_EAST', 'R10PAC_OECD', 'R10REF_ECON',
       'R10REST_ASIA', 'R10ROWO'], dtype=object)

In [312]:
selected_region = ['R10NORTH_AM', 'R10CHINA+', 'R10EUROPE', 'R10INDIA+']
count = 0
for scen in coal_data["Scenario"].unique():
    coal_data_scen = coal_data.query("Scenario==@scen")
    for model in coal_data_scen["Model"].unique():
        logistic_result = pd.DataFrame(data={"Model":np.nan,"Scenario":np.nan,"Category":np.nan,"Region":selected_region,
                                        "L":np.nan,"x0":np.nan,"k":np.nan}).set_index("Region")
        coal_data_select = coal_data_scen.query("Model==@model")
        logistic_result["Category"] = coal_data_select["Category"].unique()[0]
        logistic_result["Model"] = model
        logistic_result["Scenario"] = scen
        for reg in selected_region:
            coal_data_select_reg = coal_data_select.query("Region==@reg")
            if len(coal_data_select_reg)!= 0: # skip the regions that don't exist in this dataset
                start_year = coal_data_select_reg[coal_data_select_reg["CumCoal_GW"]==coal_data_select_reg["CumCoal_GW"].max()]["Year"].iloc[0]
                coal_start = coal_data_select_reg[coal_data_select_reg["Year"]>=start_year]
                xdata = coal_start["Year"]
                ydata = coal_start["CumCoal_GW"]
                p0 = [max(ydata), 2030,0.05]
                popt, pcov = curve_fit(f_logistic, xdata, ydata,p0, method='lm', maxfev=5000)
                logistic_result.loc[reg,"L"] = popt[0]
                logistic_result.loc[reg,"x0"] = popt[1]
                logistic_result.loc[reg,"k"] = popt[2]
                logistic_result.loc[reg,"start_year"] = start_year
                if count == 0:
                    all_logistic_result = logistic_result
                else:
                    all_logistic_result = pd.concat([all_logistic_result,logistic_result])
            count = count + 1
all_logistic_result = all_logistic_result.reset_index()
all_logistic_result["rate"] = -all_logistic_result["L"]*all_logistic_result["k"]/4
all_logistic_result["rate_percent"] = all_logistic_result["rate"]/(all_logistic_result["L"]*0.5)*100
all_logistic_result["duration"] = all_logistic_result["x0"]-all_logistic_result["start_year"]
all_logistic_result

,Region,Model,Scenario,Category,L,x0,k,start_year,rate,rate_percent,duration
0,R10NORTH_AM,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,3.722221e+02,2018.832428,0.300528,2005.0,-27.965747,-15.026375,13.832428
1,R10CHINA+,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,8.719590e+02,2024.843117,0.465561,2015.0,-101.487618,-23.278072,9.843117
2,R10EUROPE,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,R10INDIA+,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,R10NORTH_AM,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,3.722221e+02,2018.832428,0.300528,2005.0,-27.965747,-15.026375,13.832428
...,...,...,...,...,...,...,...,...,...,...,...
6047,R10INDIA+,MESSAGEix-GLOBIOM_1.1,EN_NPi2020_1000_DR3p,C3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6048,R10NORTH_AM,MESSAGEix-GLOBIOM_1.1,EN_NPi2020_1000_DR3p,C3,2.472990e+02,2033.875768,0.289880,2020.0,-17.921748,-14.493989,13.875768
6049,R10CHINA+,MESSAGEix-GLOBIOM_1.1,EN_NPi2020_1000_DR3p,C3,3.940507e+06,1974.725496,0.183738,2020.0,-181005.357165,-9.186907,-45.274504
6050,R10EUROPE,MESSAGEix-GLOBIOM_1.1,EN_NPi2020_1000_DR3p,C3,2.211853e+02,2025.438769,0.120461,2015.0,-6.661065,-6.023062,10.438769


In [340]:
for i in all_logistic_result.dropna().index:
    model = all_logistic_result.loc[i,"Model"]
    region = all_logistic_result.loc[i,"Region"]
    scenario =  all_logistic_result.loc[i,"Scenario"]
    inf_year = round(all_logistic_result.loc[i,"x0"])
    final_energy_select = final_energy.query("Model==@model").query("Region==@region").query("Scenario==@scenario").query("Year==@inf_year")
    if len(final_energy_select) != 0: 
        final_energy_value = final_energy_select["TWh/yr"].iloc[0]
        all_logistic_result.loc[i,"rate_Cherp"] = all_logistic_result.loc[i,"rate"]/final_energy_value*1000
all_logistic_result

,Region,Model,Scenario,Category,L,x0,k,start_year,rate,rate_percent,duration,rate_Cherp
0,R10NORTH_AM,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,3.722221e+02,2018.832428,0.300528,2005.0,-27.965747,-15.026375,13.832428,-7.064663
1,R10CHINA+,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,8.719590e+02,2024.843117,0.465561,2015.0,-101.487618,-23.278072,9.843117,-13.734415
2,R10EUROPE,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,R10INDIA+,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,R10NORTH_AM,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,3.722221e+02,2018.832428,0.300528,2005.0,-27.965747,-15.026375,13.832428,-7.064663
...,...,...,...,...,...,...,...,...,...,...,...,...
6047,R10INDIA+,MESSAGEix-GLOBIOM_1.1,EN_NPi2020_1000_DR3p,C3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6048,R10NORTH_AM,MESSAGEix-GLOBIOM_1.1,EN_NPi2020_1000_DR3p,C3,2.472990e+02,2033.875768,0.289880,2020.0,-17.921748,-14.493989,13.875768,-3.690181
6049,R10CHINA+,MESSAGEix-GLOBIOM_1.1,EN_NPi2020_1000_DR3p,C3,3.940507e+06,1974.725496,0.183738,2020.0,-181005.357165,-9.186907,-45.274504,NaN
6050,R10EUROPE,MESSAGEix-GLOBIOM_1.1,EN_NPi2020_1000_DR3p,C3,2.211853e+02,2025.438769,0.120461,2015.0,-6.661065,-6.023062,10.438769,-1.835377


In [341]:
all_logistic_result.set_index("Region").to_csv("AR6_log_growthRate_coal.csv")

### 5 year decline rate

In [279]:
count = 0
for scen in coal_data["Scenario"].unique():
    coal_data_scen = coal_data.query("Scenario==@scen")
    for model in coal_data_scen["Model"].unique():
        coal_data_select = coal_data_scen.query("Model==@model")
        category = coal_data_select["Category"].unique()
        for reg in selected_region:
            coal_data_select_reg = coal_data_select.query("Region==@reg")
            if len(coal_data_select_reg) != 0:
                coal_data_select_reg = coal_data_select_reg.sort_values(by="Year").set_index("Year")
                coal_data_select_reg = coal_data_select_reg.loc[:2050]
                coal_data_select_reg["Diff_GW"] = coal_data_select_reg.CumCoal_GW.diff()
                if len(coal_data_select_reg[coal_data_select_reg.Diff_GW<0]) != 0:
                    start_year = coal_data_select_reg[coal_data_select_reg.Diff_GW<0].index[0]
                    coal_data_select_reg = coal_data_select_reg[coal_data_select_reg.index>=start_year]
                    decline_result = pd.DataFrame(data={"Model": model,"Scenario":scen,"Category":category,
                                                    "Region": reg, "year":coal_data_select_reg.index[5:]},
                                              index=np.arange(len(coal_data_select_reg.index[5:]))).set_index("year")
                    for i in coal_data_select_reg.index[5:]:
                        if coal_data_select_reg.loc[i-5,"CumCoal_GW"] != 0 and coal_data_select_reg.loc[i,"CumCoal_GW"] != 0:
                            decline_result.loc[i,"growth_rate"] = np.exp(np.log(coal_data_select_reg.loc[i,"CumCoal_GW"]/coal_data_select_reg.loc[i-5,"CumCoal_GW"])/5)-1
                            decline_result.loc[i,"duration"] = i-start_year
                            decline_result.loc[i,"start year"] = start_year
                        
                if count == 0:
                    all_decline_result = decline_result
                else:
                    all_decline_result = pd.concat([all_decline_result,decline_result])
                count = count + 1

all_decline_result = all_decline_result.reset_index()
all_decline_result

,year,Model,Scenario,Category,Region,growth_rate,duration,start year
0,2011,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,R10NORTH_AM,-0.041335,5.0,2006.0
1,2012,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,R10NORTH_AM,-0.041397,6.0,2006.0
2,2013,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,R10NORTH_AM,-0.041463,7.0,2006.0
3,2014,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,R10NORTH_AM,-0.041536,8.0,2006.0
4,2015,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,R10NORTH_AM,-0.041616,9.0,2006.0
...,...,...,...,...,...,...,...,...
43455,2046,MESSAGEix-GLOBIOM_1.1,EN_NPi2020_1000_DR3p,C3,R10INDIA+,-0.117473,25.0,2021.0
43456,2047,MESSAGEix-GLOBIOM_1.1,EN_NPi2020_1000_DR3p,C3,R10INDIA+,-0.116930,26.0,2021.0
43457,2048,MESSAGEix-GLOBIOM_1.1,EN_NPi2020_1000_DR3p,C3,R10INDIA+,-0.116247,27.0,2021.0
43458,2049,MESSAGEix-GLOBIOM_1.1,EN_NPi2020_1000_DR3p,C3,R10INDIA+,-0.115365,28.0,2021.0


In [281]:
all_decline_result.set_index("Region").to_csv("AR6_5y_growthRate_coal.csv")

# Natural gas growth rate

In [282]:
gas_data = pd.read_csv("../committed_facilities/AR6_Gas_GW_R10_capacity.csv")
gas_data

,Model,Scenario,Category,Region,Year,CumGas_GW
0,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,R10NORTH_AM,2049,2.481480
1,REMIND-MAgPIE 1.7-3.0,PEP_1p5C_red_eff,C1,R10AFRICA,2024,30.233160
2,REMIND-MAgPIE 1.7-3.0,PEP_1p5C_red_eff,C1,R10CHINA+,2024,367.323220
3,REMIND-MAgPIE 1.7-3.0,PEP_1p5C_red_eff,C1,R10EUROPE,2024,251.029940
4,REMIND-MAgPIE 1.7-3.0,PEP_1p5C_red_eff,C1,R10INDIA+,2024,34.321920
...,...,...,...,...,...,...
318244,REMIND 2.1,TechCost-SSP2-B1100-ccsH,C3,R10REST_ASIA,2034,290.225260
318245,REMIND 2.1,TechCost-SSP2-B1100-ccsH,C3,R10ROWO,2034,10.379100
318246,REMIND 2.1,TechCost-SSP2-B1100-coalH,C3,R10AFRICA,2034,79.259360
318247,REMIND 2.1,TechCost-SSP2-B1100-bioH,C3,R10REF_ECON,2034,160.169180


In [291]:
count = 0
for scen in gas_data["Scenario"].unique():
    gas_data_scen = gas_data.query("Scenario==@scen")
    for model in gas_data_scen["Model"].unique():
        gas_data_select = gas_data_scen.query("Model==@model")
        category = gas_data_select["Category"].unique()
        for reg in selected_region:
            gas_data_select_reg = gas_data_select.query("Region==@reg")
            if len(gas_data_select_reg) != 0:
                gas_data_select_reg = gas_data_select_reg.sort_values(by="Year").set_index("Year")
                gas_data_select_reg = gas_data_select_reg.loc[:2050]
                growth_result = pd.DataFrame(data={"Model": model,"Scenario":scen,"Category":category,
                                                "Region": reg, "year":gas_data_select_reg.index[5:]},
                                          index=np.arange(len(gas_data_select_reg.index[5:]))).set_index("year")
                for i in gas_data_select_reg.index[5:]:
                    end_val = gas_data_select_reg.loc[i,"CumGas_GW"]
                    start_val = gas_data_select_reg.loc[i-5,"CumGas_GW"]
                    if end_val != 0 and start_val != 0:
                        growth_result.loc[i,"growth_rate"] = np.exp(np.log(end_val/start_val)/5)-1
                        
                if count == 0:
                    all_growth_result = growth_result
                else:
                    all_growth_result = pd.concat([all_growth_result,growth_result])
                count = count + 1

all_growth_result = all_growth_result.reset_index()
all_growth_result.set_index("Region").to_csv("AR6_5y_growthRate_gas.csv")
all_growth_result

,year,Model,Scenario,Category,Region,growth_rate
0,2025,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,R10NORTH_AM,-0.014495
1,2026,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,R10NORTH_AM,-0.028047
2,2027,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,R10NORTH_AM,-0.042829
3,2028,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,R10NORTH_AM,-0.059065
4,2029,REMIND-MAgPIE 2.1-4.2,CEMICS_SSP2-1p5C-fullCDR,C1,R10NORTH_AM,-0.077047
...,...,...,...,...,...,...
39333,2046,MESSAGEix-GLOBIOM_1.1,EN_NPi2020_1000_DR3p,C3,R10INDIA+,0.054438
39334,2047,MESSAGEix-GLOBIOM_1.1,EN_NPi2020_1000_DR3p,C3,R10INDIA+,0.051641
39335,2048,MESSAGEix-GLOBIOM_1.1,EN_NPi2020_1000_DR3p,C3,R10INDIA+,0.049122
39336,2049,MESSAGEix-GLOBIOM_1.1,EN_NPi2020_1000_DR3p,C3,R10INDIA+,0.046841
